In [1]:
import json

# pass.json stores the data for each iteration of our
# optimiser which created the reactor configuration we 
# expected.
with open("pass.json", "r") as f:
    pass_array = json.load(f)

# fail.json stores the data for each iteration of our
# optimiser which created the reactor configuration we 
# did not expect.
with open("fail.json", "r") as f:
    fail_array = json.load(f)

# states of the optimiser when we start to see solution divergence
pass6 = pass_array[5]
fail6 = fail_array[5]

In [2]:
import numpy as np

# show both that we have the same initial state...
assert pass6["f"] == fail6["f"]
assert np.array_equal(np.array(pass6["x"]), np.array(fail6["x"]))
assert np.array_equal(np.array(pass6["B"]), np.array(fail6["B"]))
assert np.array_equal(np.array(pass6["df"]), np.array(fail6["df"]))
assert np.array_equal(np.array(pass6["deq"]), np.array(fail6["deq"]))
assert np.array_equal(np.array(pass6["eq"]), np.array(fail6["eq"]))
assert np.array_equal(np.array(pass6["lbs"]), np.array(fail6["lbs"]))
assert np.array_equal(np.array(pass6["ubs"]), np.array(fail6["ubs"]))

In [3]:
# ... but get two different solutions to the qsp
if not np.array_equal(np.array(pass6["delta"]), np.array(fail6["delta"])):
    print("Delta arrays NOT the same")
else:
    print("Delta arrays the same")

Delta arrays NOT the same


In [4]:
f = pass6["f"]
x = np.array(pass6["x"])
B = np.array(pass6["B"])
df = np.array(pass6["df"])
deq = np.array(pass6["deq"])
eq = np.array(pass6["eq"])
lbs = np.array(pass6["lbs"])
ubs = np.array(pass6["ubs"])

In [6]:
import cvxpy as cp

# np.random.seed(42)

solutions = []

for i in range(100):
    # seeding doesn't seem to remove this non-determinism :(
    np.random.seed(42)

    # recreate `solve_qsp` in https://github.com/ukaea/PyVMCON/blob/main/src/pyvmcon/vmcon.py#L184C4-L184C4
    # where we are observing this non-determinism.
    
    delta = cp.Variable(x.shape)
    problem_statement = cp.Minimize(
        f
        + (0.5 * cp.quad_form(delta, B, assume_PSD=True))
        + (delta.T @ df)
    )

    constraints = []
    constraints.append(x + delta >= lbs)
    constraints.append(x + delta <= ubs)
    constraints.append((deq @ delta) + eq == 0)

    qsp = cp.Problem(problem_statement, constraints or None)
    qsp.solve(verbose=True, solver=cp.OSQP, eps_rel=1e-1)

    for e in solutions:
        if np.array_equal(delta.value, e):
            break
    else:
        solutions.append(delta.value)

# show flip-flop between solution seen in the 'pass' state
# and solution seen in the 'fail' state.

# Run this cell multiple times to see the proportion of `equals_pass6`
# and `equals_fail6` changes. I have never personally observed any 
# `other` solutions.
print(f"{len(solutions) = }")

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Aug 14 01:06:58 PM: Your problem has 45 variables, 3 constraints, and 0 parameters.
(CVXPY) Aug 14 01:06:58 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 14 01:06:58 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 14 01:06:58 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 14 01:06:58 PM: Compiling problem (target solver=OSQP).
(CVXPY) Aug 14 01:06:58 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing

-----------------------------------------------------------------
           OSQP v1.0.0.beta0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 45, constraints m = 116
          nnz(P) + nnz(A) = 1351
settings: algebra = Built-in,
          linear system solver = QDLDL v0.1.6,
          eps_abs = 1.0e-05, eps_rel = 1.0e-01,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          time_limit: 1.00e+10 sec,
          scaling: on, scaled_termination: off
          warm starting: on, polishing: on, 
iter   objective    prim res   dual res   rho        time
   1  -6.9544e-02   1.90e-01   5.14e+00   1.00e-01   9.97e-04s
  50   4.3991e-04   1.63e-04   1.64e-02